## Group similar products and fill NaN in the price column (Clean up brand column)

1. Created item groupings using k-means clustering
2. Calculated the mean price for all the products in the clusters (excluding products with NA)
3. Updated the price for the products with missing price with the mean price calculated above
4. Updated the brand column by:
    - converting the characters into lower case
    - replacing the punctuations with space


In [1]:
import pandas as pd
import gzip

# Reading the review
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
# Reading the image features
import struct

def readImageFeatures(path):
    f = open(path, 'rb')
    while True:
        asin = f.read(10)
        if asin == '': break
        feature = []
        for i in range(4096):
            feature.append(struct.unpack('f', f.read(4)))
        yield asin, feature

In [3]:
%cd /Users/Deepthi/Documents/DSE/Capstone/Data/From Julius

/Users/Deepthi/Documents/DSE/Capstone/Data/From Julius


In [4]:
!ls

image_features_Clothing_Shoes_and_Jewelry.b
meta_Clothing_Shoes_and_Jewelry.json.gz
ratings_Clothing_Shoes_and_Jewelry.csv
review_Women.csv
review_Women.csv.gz
review_women_brand_updated.csv
review_women_brnd_fv_prce_updated.csv
review_women_brnd_fv_updated.csv
reviews_Clothing_Shoes_and_Jewelry.json.gz
reviews_Clothing_Shoes_and_Jewelry_5.json.gz
tfidf_score.csv


In [5]:
df_meta = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')

In [6]:
df_review = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [7]:
## Julius's women's category review data
review_women = pd.read_csv('review_Women.csv')

In [8]:
## Women's category review data with modified(NA handled) brand column
review_women_brand_updated = pd.read_csv('review_women_brand_updated.csv')

In [9]:
## Women's category data with brand and feature vector updated
review_women_brnd_fv_updated = pd.read_csv('review_women_brnd_fv_updated.csv')

In [10]:
## Get the metadata with feature_vector column
df_meta_only_womens_with_fvectors = df_meta.merge(pd.DataFrame(review_women_brnd_fv_updated[['asin','feature_vector']])\
                                                  .drop_duplicates(),on='asin',how='inner')

In [11]:
df_meta_only_womens_with_fvectors.head()

,asin,categories,title,price,salesRank,imUrl,brand,related,description,feature_vector
0,B000051SEN,"[[Electronics, GPS & Navigation, Sports & Hand...","Suunto Vector Wristop Computer with Compass, A...",159.99,{u'Watches': 1647},http://ecx.images-amazon.com/images/I/51MH7ZB4...,Suunto,"{u'also_bought': [u'B00005NZWK', u'B000IU6RN6'...",NaN,0000000000000000000000000000000000000000000000...
1,B00005KJXN,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Rubies Costume Co Women's FourInOne Nun Costume,15.55,{u'Clothing': 37747},http://ecx.images-amazon.com/images/I/31ZJCQD5...,Rubie&#39;s Costume Co,"{u'also_bought': [u'B003QHZAO0', u'B000PLTLNG'...",NaN,0000000000000000000000000000000000000000000000...
2,B000074RL3,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Enell High Impact Sports Bra,NaN,{u'Sports &amp; Outdoors': 5895},http://ecx.images-amazon.com/images/I/411%2BzN...,NaN,"{u'also_bought': [u'B001BDV9SC', u'B0035WTMF4'...",The Enell High Impact Sports Bra &lt;span&gt;i...,0000000000000000000000000000000000000000000000...
3,B000078CYM,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",AmeriBag X-Small Distressed Nylon Healthy Back...,49.21,{u'Clothing': 27502},http://ecx.images-amazon.com/images/I/41ypKZPn...,AmeriBag,"{u'also_bought': [u'B001CE64C6', u'B001CE49T6'...",NaN,0000000000000000000000000000000000000000000000...
4,B00007FFL9,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",PalmBeach Jewelry Open Weave Decorative Band i...,NaN,{u'Jewelry': 76973},http://ecx.images-amazon.com/images/I/51EvJUOB...,NaN,"{u'also_bought': [u'B000X2DRRI', u'B00BRCYT98'...",NaN,0000000000000000000000000000000000000000000000...


## Clustering

In [12]:
## Cluster the products based on feature vectors

from sklearn.cluster import KMeans
import numpy as np

fv_np_array = np.array(df_meta_only_womens_with_fvectors['feature_vector'])

fv_np_array2 = []
for i in range(len(fv_np_array)):
    fv_np_array2.append(list(fv_np_array[i]))
    
print len(fv_np_array2)

## Clustered the items into 800 clusters. Can increase the number to tune the parameters
kmeans = KMeans(n_clusters=800, random_state=0).fit(fv_np_array2)

14780


In [13]:
# kmeans.labels_

In [14]:
# kmeans.predict(fv_np_array2)

In [15]:
## Create a dataframe from the output labels and name the column appropriately
clusters=pd.DataFrame(kmeans.predict(fv_np_array2))
clusters.columns=['clusters']

In [17]:
## Add the cluster label back to the original dataset
df_meta_only_womens_clusters = df_meta_only_womens_with_fvectors.join(clusters)

In [18]:
## Understand the data
df_meta_only_womens_clusters.groupby('clusters')['asin'].count()

clusters
0      10365
2          1
3          8
5         41
6        136
7        212
10       108
13        19
14        99
16       129
17        51
18        76
19       102
23        75
25        64
29        73
31       370
32        57
34        31
35        11
36         9
37         6
38        41
39        42
40         8
41        31
42        15
43        30
46        32
47        35
       ...  
333        9
336      232
337      100
341       16
349       12
366        9
371      125
385       68
391       17
395       53
405       18
421       13
429       41
439       53
450      131
472        6
479       82
518        3
534       10
537        3
551       25
557       47
569       81
590       26
591       15
607        9
653       36
656       58
673        6
699       12
Name: asin, dtype: int64

In [19]:
### Columbia Sportswear Women's Heather Canyon Wp: price is #999 in the dataset. But it is actually 99.99 so updating that

# df_meta_only_womens_clusters[df_meta_only_womens_clusters['asin']=='B004LP2OJU']

df_meta_only_womens_clusters.set_value(5404, 'price', 99.99)

,asin,categories,title,price,salesRank,imUrl,brand,related,description,feature_vector,clusters
0,B000051SEN,"[[Electronics, GPS & Navigation, Sports & Hand...","Suunto Vector Wristop Computer with Compass, A...",159.99,{u'Watches': 1647},http://ecx.images-amazon.com/images/I/51MH7ZB4...,Suunto,"{u'also_bought': [u'B00005NZWK', u'B000IU6RN6'...",NaN,0000000000000000000000000000000000000000000000...,0
1,B00005KJXN,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Rubies Costume Co Women's FourInOne Nun Costume,15.55,{u'Clothing': 37747},http://ecx.images-amazon.com/images/I/31ZJCQD5...,Rubie&#39;s Costume Co,"{u'also_bought': [u'B003QHZAO0', u'B000PLTLNG'...",NaN,0000000000000000000000000000000000000000000000...,0
2,B000074RL3,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Enell High Impact Sports Bra,NaN,{u'Sports &amp; Outdoors': 5895},http://ecx.images-amazon.com/images/I/411%2BzN...,NaN,"{u'also_bought': [u'B001BDV9SC', u'B0035WTMF4'...",The Enell High Impact Sports Bra &lt;span&gt;i...,0000000000000000000000000000000000000000000000...,0
3,B000078CYM,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",AmeriBag X-Small Distressed Nylon Healthy Back...,49.21,{u'Clothing': 27502},http://ecx.images-amazon.com/images/I/41ypKZPn...,AmeriBag,"{u'also_bought': [u'B001CE64C6', u'B001CE49T6'...",NaN,0000000000000000000000000000000000000000000000...,0
4,B00007FFL9,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",PalmBeach Jewelry Open Weave Decorative Band i...,NaN,{u'Jewelry': 76973},http://ecx.images-amazon.com/images/I/51EvJUOB...,NaN,"{u'also_bought': [u'B000X2DRRI', u'B00BRCYT98'...",NaN,0000000000000000000000000000000000000000000000...,0
5,B00007GDD3,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Double-Flap Mini-Bag with Total Wallet ...,18.37,NaN,http://ecx.images-amazon.com/images/I/41EHat-F...,Buxton,"{u'also_bought': [u'B003E76ME4', u'B0048991OU'...",NaN,0000000000000000000000000000000000000000000000...,0
6,B00007GDAL,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Double CardexTM,19.99,NaN,http://ecx.images-amazon.com/images/I/41ENA0kC...,Buxton,"{u'also_bought': [u'B00021Q2JU', u'B008E7TQYC'...",NaN,0000000000000000000000000000000000000000000000...,0
7,B00007GD9I,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Heiress Organizer; Clutch,26.47,NaN,http://ecx.images-amazon.com/images/I/41cga2O2...,Buxton,"{u'also_bought': [u'B00JXRBM1C', u'B00CYR6EX4'...",NaN,0000000000000000000000000000000000000000000000...,0
8,B00007GD8X,"[[Clothing, Shoes & Jewelry, Women, Handbags &...",Buxton Check Clutch Mini Bag On A String,13.64,NaN,http://ecx.images-amazon.com/images/I/41oJPUrC...,Buxton,"{u'also_bought': [u'B004C51NRS', u'B003E76ME4'...",NaN,0000000000000000000000000000000000000000000000...,0
9,B00007GD9W,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Buxton Heiress Convertible&reg; Billfold,18.02,{u'Clothing': 28110},http://ecx.images-amazon.com/images/I/41olO1DM...,Buxton,"{u'also_bought': [u'B00523T0TC', u'B000E83VU4'...",NaN,0000000000000000000000000000000000000000000000...,0


In [20]:
## For exploratory purposes
df_meta_only_womens_clusters[df_meta_only_womens_clusters['clusters']==16]

,asin,categories,title,price,salesRank,imUrl,brand,related,description,feature_vector,clusters
938,B000LWKD5E,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Naturalizer Women's Lulianna Wedge Sandal,NaN,{u'Shoes': 39818},http://ecx.images-amazon.com/images/I/51rZlkvt...,NaN,"{u'also_bought': [u'B005CNI80O', u'B008HC4OJ6'...",NaN,0000000000000100000000000000000000000000000000...,16
961,B000MKKS5U,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Naot Women's Kayla Wedge Sandal,NaN,{u'Shoes': 10381},http://ecx.images-amazon.com/images/I/416m1DLP...,NaN,"{u'also_bought': [u'B00L5FXDU8', u'B00L5FXHD6'...",NaN,0000000000000100000000000000000000000000000000...,16
1666,B0012SC9HG,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Lauren Ralph Lauren Women's Cecilia Wedge Sandal,NaN,{u'Shoes': 350},http://ecx.images-amazon.com/images/I/41wCUZPZ...,NaN,"{u'also_bought': [u'B00BG5XBZE', u'B00FOQ6BGC'...",NaN,0000000000000100000000000000000000000000000000...,16
1956,B001AZJVPO,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Crocs Women's Sassari Wedge Sandal,NaN,{u'Shoes': 82300},http://ecx.images-amazon.com/images/I/41XlpXuL...,NaN,"{u'also_bought': [u'B006VB4LRU', u'B004IZ2XMQ'...",NaN,0000000000000100000000000000000000000000000000...,16
2175,B001HS2W46,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Reef Women's Krystal Star Wedge Sandal,NaN,{u'Shoes': 912},http://ecx.images-amazon.com/images/I/41OM3jdF...,NaN,"{u'also_bought': [u'B00BKTPLM2', u'B00BKTPLNQ'...",NaN,0000000000000100000000000000000000000000000000...,16
2432,B001T0IM5U,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Aerosoles Women's Plush Around Wedge Sandal,NaN,{u'Shoes': 4453},http://ecx.images-amazon.com/images/I/41uex-Ye...,NaN,"{u'also_bought': [u'B006Z6H9AM', u'B0033WTKZI'...",NaN,0000000000000100000000000000000000000000000000...,16
2970,B002JCSTWI,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...","Crocs Women's Farrah Wedge Sandal,Gold/Oyster,...",NaN,{u'Shoes': 81375},http://ecx.images-amazon.com/images/I/41m%2BTt...,NaN,{u'also_viewed': [u'B008XHA9DK']},NaN,0000000000000100000000000000000000000000000000...,16
3065,B002N5NFOI,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Crocs Womens Patricia Wedge Sandal,NaN,{u'Shoes': 4544},http://ecx.images-amazon.com/images/I/411SG2R3...,NaN,"{u'also_bought': [u'B001IV5BW2', u'B00CN7RYJI'...",NaN,0000000000000100000000000000000000000000000000...,16
3235,B002UD4UQU,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...","Skechers Cali Women's Meow Wedge Sandal,Black,...",NaN,{u'Shoes': 83255},http://ecx.images-amazon.com/images/I/41d5rwIP...,NaN,"{u'also_viewed': [u'B005LLDMHQ', u'B00CCLJCCC'...",NaN,0000000000000100000000000000000000000000000000...,16
3583,B0036XNGY0,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",Skechers Women's Rumblers-Home Run Wedge Sandal,NaN,{u'Shoes': 1590},http://ecx.images-amazon.com/images/I/41Znwoy7...,NaN,"{u'also_bought': [u'B001OQBN96', u'B004CLZE22'...",NaN,0000000000000100000000000000000000000000000000...,16


In [21]:
## Calculate mean price for each cluster (consider clusters without NANs)

df_cluster_mean_price = pd.DataFrame(df_meta_only_womens_clusters[df_meta_only_womens_clusters['price'].notnull()]\
                        .groupby('clusters').price.mean()).reset_index()

df_cluster_mean_price.columns=['clusters','mean_price']
# .to_csv('validate_price.csv')

In [22]:
## Merge the mean price back to original data
df_meta_only_womens_clusters = df_meta_only_womens_clusters.merge(df_cluster_mean_price,on='clusters')

In [23]:
df_meta_only_womens_clusters.head()

,asin,categories,title,price,salesRank,imUrl,brand,related,description,feature_vector,clusters,mean_price
0,B000051SEN,"[[Electronics, GPS & Navigation, Sports & Hand...","Suunto Vector Wristop Computer with Compass, A...",159.99,{u'Watches': 1647},http://ecx.images-amazon.com/images/I/51MH7ZB4...,Suunto,"{u'also_bought': [u'B00005NZWK', u'B000IU6RN6'...",NaN,0000000000000000000000000000000000000000000000...,0,26.23434
1,B00005KJXN,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Rubies Costume Co Women's FourInOne Nun Costume,15.55,{u'Clothing': 37747},http://ecx.images-amazon.com/images/I/31ZJCQD5...,Rubie&#39;s Costume Co,"{u'also_bought': [u'B003QHZAO0', u'B000PLTLNG'...",NaN,0000000000000000000000000000000000000000000000...,0,26.23434
2,B000074RL3,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",Enell High Impact Sports Bra,NaN,{u'Sports &amp; Outdoors': 5895},http://ecx.images-amazon.com/images/I/411%2BzN...,NaN,"{u'also_bought': [u'B001BDV9SC', u'B0035WTMF4'...",The Enell High Impact Sports Bra &lt;span&gt;i...,0000000000000000000000000000000000000000000000...,0,26.23434
3,B000078CYM,"[[Clothing, Shoes & Jewelry, Shoes & Accessori...",AmeriBag X-Small Distressed Nylon Healthy Back...,49.21,{u'Clothing': 27502},http://ecx.images-amazon.com/images/I/41ypKZPn...,AmeriBag,"{u'also_bought': [u'B001CE64C6', u'B001CE49T6'...",NaN,0000000000000000000000000000000000000000000000...,0,26.23434
4,B00007FFL9,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",PalmBeach Jewelry Open Weave Decorative Band i...,NaN,{u'Jewelry': 76973},http://ecx.images-amazon.com/images/I/51EvJUOB...,NaN,"{u'also_bought': [u'B000X2DRRI', u'B00BRCYT98'...",NaN,0000000000000000000000000000000000000000000000...,0,26.23434


In [24]:
## Create a new column with NaNs replaced with Price
df_meta_only_womens_clusters['price_new']=df_meta_only_womens_clusters.price\
                                          .fillna(df_meta_only_womens_clusters.mean_price)
    
df_updated_price = df_meta_only_womens_clusters[['asin','price_new']].drop_duplicates()

In [25]:
## Merge it back into the review_women_brnd_fv_updated data

review_women_brnd_fv_prce_updated = review_women_brnd_fv_updated.merge(df_updated_price,on='asin',how='left')\
                                   [['reviewerID','asin','unixReviewTime','brand','price_new','feature_vector']]
    
review_women_brnd_fv_prce_updated.columns=['reviewerID','asin','unixReviewTime','brand','price','feature_vector']

### Update the brand field

In [26]:
import string
import re

## Replace punctuations with space

RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  

review_women_brnd_fv_prce_updated['brand']=review_women_brnd_fv_prce_updated['brand'].str.lower()\
                                            .str.replace(RE_PUNCTUATION, " ")

In [27]:
## Export to CSV
review_women_brnd_fv_prce_updated.to_csv('review_women_brnd_fv_prce_updated.csv',index=False)

In [28]:
review_women_brnd_fv_prce_updated.head()

,reviewerID,asin,unixReviewTime,brand,price,feature_vector
0,A4KU0XJNBH674,B000051SEN,1372723200,suunto,159.99,0000000000000000000000000000000000000000000000...
1,A3HY330W94JPUQ,B000051SEN,1354492800,suunto,159.99,0000000000000000000000000000000000000000000000...
2,A1CHOKV10NEI8X,B000051SEN,1143936000,suunto,159.99,0000000000000000000000000000000000000000000000...
3,AOUBIY0S651IU,B000051SEN,1399248000,suunto,159.99,0000000000000000000000000000000000000000000000...
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,suunto,159.99,0000000000000000000000000000000000000000000000...
